<a href="https://colab.research.google.com/github/Ronbragaglia/ColabPython/blob/main/Resist%C3%AAnciaConcreto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U scikit-learn
!pip install -U pandas
!pip install -U xlrd
!pip install -U requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 20.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import requests
import zipfile
import random
import io

from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error
from pathlib import Path

In [ ]:
url = "https://archive.ics.uci.edu/static/public/165/concrete+compressive+strength.zip"

response = requests.get(url)

if response.ok:
    zip_content = io.BytesIO(response.content)

    with zipfile.ZipFile(zip_content, "r") as zip_ref:
        zip_ref.extractall("concrete_compressive_strength")
else:
    print("STATUS CODE != 200")

In [ ]:
def load_dataset(dataset_path: Path, train_percent=10) -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """Lê o datasets para o treinamento & teste do modelo.

    :param dataset_path: Diretório do arquivo contendo o dataset.
    :param train_percent: Porcentagem de dados que será tirado do dataset para testagem do modelo.
    :return: Quatro valores do tipo ndarray na seguinte sequencia
    """

    dataset = pd.read_excel(dataset_path)

    train_quant = int(len(dataset)*(train_percent/100))-4

    random.seed(4)
    rand_pick = random.randint(0, len(dataset)-train_quant)

    pick_indices = [rand_pick, rand_pick+train_quant]

    train_data = dataset.iloc[pick_indices[0]:pick_indices[1]]

    dataset.drop(dataset.index[pick_indices[0]:pick_indices[1]], inplace=True)

    X_train = dataset.iloc[:, :-1]
    y_train = dataset.iloc[:, -1:]
    X_test = train_data.iloc[:, :-1]
    y_test = train_data.iloc[:, -1:]

    return X_train, y_train, X_test, y_test

In [ ]:
X_train, y_train, X_test, y_test = load_dataset(dataset_path=Path(r"concrete_compressive_strength/Concrete_Data.xls"), train_percent=10)

In [ ]:
model = MLPRegressor(hidden_layer_sizes=(20, 4),
                      learning_rate_init=0.1,
                      max_iter=1000,
                      random_state=42,
                      momentum=0.01)

model.fit(X_train, y_train.values.ravel())

MLPRegressor(hidden_layer_sizes=(20, 4), learning_rate_init=0.1, max_iter=1000,
             momentum=0.01, random_state=42)

In [ ]:
y_pred = model.predict(X_test)
print(f"Mean Absolute Error: {mean_absolute_error(y_test, y_pred):>20}")

idx = random.randint(0, len(X_test)-1)
print(f"Portando, o concreto quando feito com essa quantidade de elementos terá uma resistência à pressão de aproximadamente {model.predict(X_test[idx - 1:idx:])} megapascals (MPa) - valor predito pelo modelo -.")
print("Valor verdadeiro que está nos dados de teste: ", y_test[idx - 1:idx:].values)
X_test[idx - 1:idx:]

Mean Absolute Error:    4.721312625240598
Portando, o concreto quando feito com essa quantidade de elementos terá uma resistência à pressão de aproximadamente [48.75596166] megapascals (MPa) - valor predito pelo modelo -.
Valor verdadeiro que está nos dados de teste:  [[43.58177796]]


,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day)
278,251.37,0.0,118.27,188.45,5.75,1028.4,757.73,100


Através dos parâmetros de entradas e de saída contruímos, treinamos e testamos a RNA_MLP de Regressão para encontrar a resistência correta do concreto com base nos parâmetros quantitativos de entrada, conforme segue:

- Número de instâncias: 1030

- Número de atributos: 9

- Divisão de atributos: 8 variáveis ​​de entrada quantitativas (Cimento, Escória de alto-forno, Cinza volante, Água, Superplastificante, Agregados graúdo, Agregados fino, Idade) e 1 variável de saída quantitativa (Resistência do Concreto).

- Foram parametrizados 20 portas de entradas e 4 portas ocultas no teste realizado.

- Taxa de Aprendizagem: 95%

- Número máximo de épocas: 1000

No código fornecido para rastrear o número de épocas para atingir a convergência pode se usar um callback ,como  "sklearn.callbacks.EarlyStopping" que interrompe o treinamento quanto uma determinada métrica para de funcionar.

O scikit -learn , na versão atual não fornece uma maneira direta de acessar pesos finais de um modelo treinado com MLPREGRESSOR.

Acurácia dos testes foi de 95%

Resultados obtidos para os valores abaixo:

- Contendo os seguintes elementos:
- Cimento (1kg/m³ de mistura): 278
- Escória de alto-forno (2kg/m³ de mistura ): 251,37
- Cinza volante (3kg/m³ de mistura): 0
- Água (4kg/m³ de mistura): 118,27
- Superplastificante (5kg/m³ de mistura): 188,45
- Agregados graúdo (6kg/m³ de mistura): 5.75
- Agregados miúdo (7kg/m³ de mistura): 1028,4
- Idade (dias): 100

Portando, segundo a predição do modelo, o concreto quando feito com essa quantidade de elementos terá uma resistência à pressão de aproximadamente 48.76 MPa aos 100 dias de idade.


Valor predito pelo modelo de regressão: 48.76 MPa

Valor verdadeiro que está nos dados de teste: 43.58 MPa
